In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#test file
path = "/content/drive/MyDrive/dataset/test.csv"
df_test = pd.read_csv(path, escapechar="\\", quoting=csv.QUOTE_NONE, error_bad_lines=False)

# df_test = pd.read_csv(path, escapechar = "\\", quoting = csv.QUOTE_NONE)
df_test.head()

<ipython-input-5-9a2eae260a77>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_test = pd.read_csv(path, escapechar="\\", quoting=csv.QUOTE_NONE, error_bad_lines=False)
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

<ipython-input-5-9a2eae260a77>:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(path, escapechar="\\", quoting=csv.QUOTE_NONE, error_bad_lines=False)


,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142.0
1,938007,Pokemon M-037 Kyurem Figure Monster Collection...,NaN,NaN,0.0
2,708128,Ganz normal verrückt: Das Wassermannzeitalter ...,NaN,NaN,10512.0
3,500777,Notebook: Japanese Anime Lovers Gift Manga Sen...,NaN,NaN,12415.0
4,2217122,Mon journal intime licorne: Idée de cadeau pou...,NaN,NaN,1.0


In [7]:
#train file
path = "/content/drive/MyDrive/dataset/train.csv"
df_train = pd.read_csv(path, escapechar="\\", quoting=csv.QUOTE_NONE, error_bad_lines=False)

# df_train = pd.read_csv(path, escapechar = "\\", quoting = csv.QUOTE_NONE)
df_train = df_train.dropna()
df_train.head()

Streaming output truncated to the last 5000 lines.
Skipping line 551779: expected 55 fields, saw 78
Skipping line 551947: expected 55 fields, saw 59
Skipping line 552053: expected 55 fields, saw 67
Skipping line 552066: expected 55 fields, saw 70
Skipping line 552070: expected 55 fields, saw 107
Skipping line 552129: expected 55 fields, saw 72
Skipping line 552182: expected 55 fields, saw 59
Skipping line 552206: expected 55 fields, saw 74
Skipping line 552217: expected 55 fields, saw 63
Skipping line 552329: expected 55 fields, saw 58
Skipping line 552396: expected 55 fields, saw 64
Skipping line 552486: expected 55 fields, saw 102
Skipping line 552806: expected 55 fields, saw 64
Skipping line 552960: expected 55 fields, saw 68
Skipping line 553050: expected 55 fields, saw 70
Skipping line 553084: expected 55 fields, saw 85
Skipping line 553107: expected 55 fields, saw 62
Skipping line 553131: expected 55 fields, saw 68
Skipping line 553364: expected 55 fields, saw 62
Skipping line 55

,,,,,,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
1329311,"""Amscan 354001.54 Jumbo Margarita Glass",30 oz,"Caribbean Blue""","""[30 oz.",High quality paper,Drink in style with this plastic margarita glass,Perfect for parties,celebrations and adds a classy and simple acc...,Serve cool drinks in our 30 oz Jumbo Plastic M...,1419.0,950.0
798322,"""K-Swiss 201 Classic Tennis Shoe (Infant/Toddler)",Black/Black,"5 M US Toddler""","""[Sleek low-top sneaker with vertically stitched toe",D-ring hardware lacing,"Perforated tongue]""","""K-Swiss 201 Classic Tennis Shoe (Infant/Toddler)",Black/Black,"5 M US Toddler""",2449.0,750.0
1629663,"""Liakada Girls Argyle Supportive Sports Bra with Scoop Neckline Dance",Gym,Yoga,"Cheer! Black""","""[This sports bra top features contract criss-cross straps on the back and a scoop neckline for a clean finish.",Integrated bra shelf liner,Adjustable straps in a woven pattern,"92% Nylon 8% Spandex]""",This sports bra top features contract criss-cr...,2640.0,800.0
1833730,Khadims Men Black Slip-On Shoe - UK 9,"""[Upper Material: Synthetic / Sole Material: Airmax",Occasion: Formal / Utility: Office Formals,Closure: Slip-on / Toe Style: Closed,Shoe Style: Slip-On / Shoe Type: Shoe,Care Instructions: 1. Use a soft shoe brush to...,"""Khadim’s combines comfort and durability with...",a pair of Khadim’s on your feet gives you tha...,Khadim’s gives you an unmatched value-for-mon...,3246.0,1279.527558
2656446,TONYBOY Boy's Yarn Dyed Checks Shirt & Solid Shirt - Green/Fluo Orange (Pack of 2),"""[Fit Type: Regular Fit",Material: Cotton Fabric,Pattern : 1pcs. Y/D Checks Shirt || 1 pc. Solid Shirt,Sleeve: Full Sleeve || Neck: Regular Collar,"Wash Care: Wash Dark Colour Separately]""","""Tonyboy Brand Presents The Combo shirt of Yar...",The Shirt Is Fit To Be Worn In Casual & Forma...,"Shoes & Sneakers For A Cool Smart Look.""",2677.0,984.251967


In [8]:
punctuation_signs = list("?:!.,;")
nltk.download('punkt')
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
df_train['TITLE'] = df_train['TITLE'].str.replace("\r", " ")
df_train['TITLE'] = df_train['TITLE'].str.replace("\n", " ")
df_train['TITLE'] = df_train['TITLE'].str.replace("    ", " ")
df_train['TITLE'] = df_train['TITLE'].str.replace('"', '')
df_train['TITLE'] = df_train['TITLE'].str.lower()
for punct_sign in punctuation_signs:
  df_train['TITLE'] = df_train['TITLE'].str.replace(punct_sign, '')
df_train['TITLE'] = df_train['TITLE'].str.replace("'s", "")

<ipython-input-9-a6dabd7cd60f>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train['TITLE'] = df_train['TITLE'].str.replace(punct_sign, '')


In [10]:
final_cols = ["TITLE", "PRODUCT_LENGTH"]
df_train = df_train[final_cols]
df_train = df_train.iloc[:35000, :]

In [11]:
df_test['TITLE'] = df_test['TITLE'].str.replace("\r", " ")
df_test['TITLE'] = df_test['TITLE'].str.replace("\n", " ")
df_test['TITLE'] = df_test['TITLE'].str.replace("    ", " ")
df_test['TITLE'] = df_test['TITLE'].str.replace('"', '')
df_test['TITLE'] = df_test['TITLE'].str.lower()
for punct_sign in punctuation_signs:
  df_test['TITLE'] = df_test['TITLE'].str.replace(punct_sign, '')
df_test['TITLE'] = df_test['TITLE'].str.replace("'s", "")

<ipython-input-11-714973f78bc8>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_test['TITLE'] = df_test['TITLE'].str.replace(punct_sign, '')


In [12]:
final_cols = ["TITLE", "PRODUCT_ID"]
df_test = df_test[final_cols]

In [13]:
df_train.head()

,,,,,,TITLE,PRODUCT_LENGTH
1329311,"""Amscan 354001.54 Jumbo Margarita Glass",30 oz,"Caribbean Blue""","""[30 oz.",High quality paper,perfect for parties,950.0
798322,"""K-Swiss 201 Classic Tennis Shoe (Infant/Toddler)",Black/Black,"5 M US Toddler""","""[Sleek low-top sneaker with vertically stitched toe",D-ring hardware lacing,k-swiss 201 classic tennis shoe (infant/toddler),750.0
1629663,"""Liakada Girls Argyle Supportive Sports Bra with Scoop Neckline Dance",Gym,Yoga,"Cheer! Black""","""[This sports bra top features contract criss-cross straps on the back and a scoop neckline for a clean finish.",adjustable straps in a woven pattern,800.0
1833730,Khadims Men Black Slip-On Shoe - UK 9,"""[Upper Material: Synthetic / Sole Material: Airmax",Occasion: Formal / Utility: Office Formals,Closure: Slip-on / Toe Style: Closed,Shoe Style: Slip-On / Shoe Type: Shoe,khadim’s combines comfort and durability with ...,1279.527558
2656446,TONYBOY Boy's Yarn Dyed Checks Shirt & Solid Shirt - Green/Fluo Orange (Pack of 2),"""[Fit Type: Regular Fit",Material: Cotton Fabric,Pattern : 1pcs. Y/D Checks Shirt || 1 pc. Solid Shirt,Sleeve: Full Sleeve || Neck: Regular Collar,tonyboy brand presents the combo shirt of yarn...,984.251967


In [14]:
df_test.head()

,,,,,,TITLE,PRODUCT_ID
1925202,"""ArtzFolio Tulip Flowers Blackout Curtain for Door","Window & Room | Eyelets & Tie Back | Canvas Fabric | Width 4.5feet (54inch) Height 5 feet (60 inch); Set of 2 PCS""","""[LUXURIOUS & APPEALING: Beautiful custom-made curtains to decorate any home or office | Includes inbuilt tieback to hold the curtain | Completely finished and ready to hang on walls & windows",MATERIAL: Luxurious & versatile fabric with a natural finish | High colour fastness | State-of-the-art digital printing ensures colour consistency and prevents any fading | Eyelets; Cotton Canvas; Width 4.5feet (54inch) | Multicolour | PACKAGE: 2 Room Curtains Eyelets | SIZE: Height 5 feet (60 inch); SET OF 2 PCS,BLACKOUT CURTAIN: 100% opaque & heavy premium cotton canvas fabric | Tight knitted,made to perfection large eyelets at the top to...,long life & durable fabric | Printing only on...
2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_Navy Mix_9-10Y,"""[Harry Potter Hedwig Pyjamas (6-16 Yrs)",100% cotton (exclusive of trimmings),Ribbing - 98% cotton,"2% elastane]""",2755,NaN
283658,The United Empire Loyalists: A Chronicle of the Great Migration,NaN,NaN,6112,598.424,NaN,NaN
413758,Ungifted: My Life and Journey,NaN,NaN,23,598.0,NaN,NaN
2050239,PUMA Cali Sport Clean Women's Sneakers White Leather (37540701),"""[Style Name:-Cali Sport Clean Women's Sneakers",Upper Material:-Leather,Lower Material:-Rubber,Care Instructions: Wipe with a clean dry cloth,NaN,"Warranty Period: 90 Days Manufacturers Warranty]"""


In [15]:
df_train.isna().sum()

TITLE             0
PRODUCT_LENGTH    0
dtype: int64

In [17]:
df_test["TITLE"].fillna("No Data", inplace = True)

In [18]:
df_test.isna().sum()

TITLE              0
PRODUCT_ID    727085
dtype: int64

In [21]:
X_train, X_test, y_train = df_train["TITLE"], df_test["TITLE"], df_train["PRODUCT_LENGTH"]

In [22]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=5)
print('1')
X_train_vectors_tfidf = tfidf.fit_transform(X_train)
print(X_train_vectors_tfidf.shape)
print('1')
X_test_vectors_tfidf = tfidf.transform(X_test)
print(X_test_vectors_tfidf.shape)

1
(35000, 9250)
1
(2190214, 9250)


In [26]:
import numpy as np

# convert y_train to a list of strings
y_train_str = y_train.astype(str).tolist()

# check the unique values of y_train_str
print(np.unique(y_train_str))



[' ' ' ""I love your story and your message'
 ' ""If You Could See Me Now""' ...
 '🧸 [Durable]This premium quality cargo net will hold cargo securely in place as you drive.'
 '🧼 [Best Gift]: Practical and classy'
 '🧿 Wireless Charging: SAMEO Airpods Case has Easily Charge. The Waterproof Sealing Plug at the Bottom makes it Easy to Access the Lightning Port']


In [ ]:
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train_str)

In [ ]:
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)

In [ ]:
y_predict

In [ ]:
len(y_predict)

In [ ]:
d = {"PRODUCT_LENGTH" : y_predict}

In [ ]:
df1 = pd.DataFrame(data=d)
df1

In [ ]:
df_new = pd.concat([df_test, df1], axis = 1)
df_new.head()

In [ ]:
l = ["PRODUCT_ID", "PRODUCT_LENGTH"]
df_new = df_new[l]
df_new

In [ ]:
df_new.to_csv("/content/drive/MyDrive/submission.csv", index = False, header = True)

In [ ]:
path = "/content/drive/MyDrive/submission.csv"
df_sub = pd.read_csv(path)
df_sub.head()

In [ ]:
df_sub.isna().sum()

In [ ]:
df_sub.describe()